In [5]:
import requests
import os
import uuid
import time
import sys
from dotenv import load_dotenv

# --- LISTA DAS 22 CONTAS PROBLEMA ---
ALVOS = [
    "004769828", "001970006", "004380499", "001876856", "001970043",
    "021677521", "002375936", "001648477", "008694201", "005030566",
    "001174962", "004839388", "001295850", "009061265", "004030265",
    "004689408", "003697008", "004920099", "004627337", "004957007",
    "009177993", "004801297"
]

# --- 1. LÓGICA DE CARREGAMENTO DO .ENV (IGUAL AO PRINCIPAL) ---
# Tenta carregar normal
load_dotenv()
CLIENT_ID = os.getenv("btg_client_id")
CLIENT_SECRET = os.getenv("btg_client_secret")
WEBHOOK_TOKEN = os.getenv("WEBHOOK_TOKEN")

# Se falhar, tenta o truque do UTF-16 (que o script principal usa)
if not CLIENT_ID:
    load_dotenv(encoding='utf-16')
    CLIENT_ID = os.getenv("btg_client_id") or os.getenv("BTG_CLIENT_ID")
    CLIENT_SECRET = os.getenv("btg_client_secret") or os.getenv("BTG_CLIENT_SECRET")
    WEBHOOK_TOKEN = os.getenv("WEBHOOK_TOKEN")

# Validação final
if not CLIENT_ID or not CLIENT_SECRET or not WEBHOOK_TOKEN:
    print("ERRO CRÍTICO: Não foi possível ler as variáveis do .env (Mesmo tentando UTF-16)")
    print(f"Status atual -> ID: {CLIENT_ID is not None} | Secret: {CLIENT_SECRET is not None} | Token: {WEBHOOK_TOKEN is not None}")
    sys.exit()

WEBHOOK_URL = f"https://weebhook-btg.onrender.com/webhook?token={WEBHOOK_TOKEN}"

# --- 2. FUNÇÃO DE TOKEN (IGUAL AO PRINCIPAL) ---
def get_token(client_id, client_secret):
    url = "https://api.btgpactual.com/iaas-auth/api/v1/authorization/oauth2/accesstoken"
    headers = {
        'x-id-partner-request': str(uuid.uuid4()),
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json'
    }
    data = {'grant_type': 'client_credentials'}
    
    try:
        # Usa auth tuple para garantir a codificação correta
        r = requests.post(url, data=data, headers=headers, auth=(client_id, client_secret))
        
        if r.status_code == 200:
            # Tenta pegar do header (padrão BTG antigo) ou do JSON (padrão novo)
            token = r.headers.get('access_token')
            if not token:
                token = r.json().get('access_token')
            return token
        else:
            print(f"ERRO AO GERAR TOKEN: Status {r.status_code} - {r.text}")
            return None
    except Exception as e:
        print(f"ERRO DE CONEXAO NO TOKEN: {e}")
        return None

# --- 3. FUNÇÃO DE INVESTIGAÇÃO ---
def investigar_conta(token, conta):
    url = "https://api.btgpactual.com/iaas-profitability/api/v1/performance-report/account"
    
    # Formatação de segurança
    conta_envio = str(conta).strip().zfill(9)
    req_id = str(uuid.uuid4())
    
    body = {
        "accountNumber": conta_envio,
        "startDate": "2026-01-01",
        "endDate": "2026-01-31",
        "webhookService": "performance-report",
        "callbackUrl": WEBHOOK_URL 
    }
    
    print(f"Conta {conta_envio}...", end=" ")
    
    try:
        r = requests.post(
            url, 
            headers={'access_token': token, 'x-id-partner-request': req_id}, 
            json=body, 
            timeout=15
        )
        
        if r.status_code in [200, 202]:
            print("✅ SUCESSO (Aceitou)")
        elif r.status_code == 429:
            print("⏳ Rate Limit (429) - Esperando...")
            time.sleep(30)
        else:
            # AQUI VAI SAIR O MOTIVO DO ERRO
            print(f"❌ REJEITADO ({r.status_code}) -> {r.text}")
            
    except Exception as e:
        print(f"Erro de request: {e}")

# --- EXECUÇÃO ---
if __name__ == "__main__":
    print(f"--- DIAGNÓSTICO DE {len(ALVOS)} CONTAS ---")
    
    # Usa as credenciais que carregamos lá em cima
    token = get_token(CLIENT_ID, CLIENT_SECRET)
    
    if token:
        print(f"Token gerado com sucesso! (Início: {token[:10]}...)")
        print("-" * 50)
        
        for conta in ALVOS:
            investigar_conta(token, conta)
            time.sleep(1.2) # Respeita o limite da API
    else:
        print("Falha ao obter token. Verifique se o .env está na mesma pasta.")

--- DIAGNÓSTICO DE 22 CONTAS ---
Token gerado com sucesso! (Início: eyJhbGciOi...)
--------------------------------------------------
Conta 004769828... ✅ SUCESSO (Aceitou)
Conta 001970006... ✅ SUCESSO (Aceitou)
Conta 004380499... ✅ SUCESSO (Aceitou)
Conta 001876856... ✅ SUCESSO (Aceitou)
Conta 001970043... ✅ SUCESSO (Aceitou)
Conta 021677521... ✅ SUCESSO (Aceitou)
Conta 002375936... ✅ SUCESSO (Aceitou)
Conta 001648477... ✅ SUCESSO (Aceitou)
Conta 008694201... ✅ SUCESSO (Aceitou)
Conta 005030566... ✅ SUCESSO (Aceitou)
Conta 001174962... ✅ SUCESSO (Aceitou)
Conta 004839388... ✅ SUCESSO (Aceitou)
Conta 001295850... ✅ SUCESSO (Aceitou)
Conta 009061265... ✅ SUCESSO (Aceitou)
Conta 004030265... ✅ SUCESSO (Aceitou)
Conta 004689408... ✅ SUCESSO (Aceitou)
Conta 003697008... ✅ SUCESSO (Aceitou)
Conta 004920099... ✅ SUCESSO (Aceitou)
Conta 004627337... ✅ SUCESSO (Aceitou)
Conta 004957007... ✅ SUCESSO (Aceitou)
Conta 009177993... ✅ SUCESSO (Aceitou)
Conta 004801297... ✅ SUCESSO (Aceitou)
